- MLP
- Multiple countries
- Trained on the whole sequence, not autoregressive

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad

In [2]:
# Load the CSV file
path = "data/country_wise_data.csv"
df = pd.read_csv(path)  # Replace with your file path

country = 'US'
kind = 'New cases' #[Confirmed, Deaths,	Recovered, Active, New cases, New deaths, New recovered]

# Filter rows for the coutry in the "Country/Region" column
country_data = df[df["Country/Region"] == country]
country_data["Date"] = pd.to_datetime(country_data["Date"])
country_data["Date"] = (country_data["Date"] - country_data["Date"].iloc[0]).dt.days

# Find the index of the first non-zero value in the "New cases" column
first_non_zero_index = country_data[country_data[kind] != 0].index[0]
# Slice the DataFrame to start from the first non-zero row
country_data = country_data.loc[first_non_zero_index:]
MAX = max(country_data[kind])
country_data[kind] = country_data[kind]/MAX

# Extract time (e.g., "Date") and target (e.g., "New cases") columns
time_data = torch.tensor(country_data["Date"].values, dtype=torch.float32).view(-1, 1)
target_data = torch.tensor(country_data["New cases"].values, dtype=torch.float32).view(-1, 1)

C:\Users\hanky\AppData\Local\Temp\ipykernel_31268\799233015.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_data["Date"] = pd.to_datetime(country_data["Date"])
C:\Users\hanky\AppData\Local\Temp\ipykernel_31268\799233015.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_data["Date"] = (country_data["Date"] - country_data["Date"].iloc[0]).dt.days


In [ ]:
countries = {"Republic of Korea": {1: [20, 211], 2: [205, 295], 3: [295, 436]},
            }

In [3]:
def plot_predictions(t_data, y_data, t_train, y_train, t_predict, predicted_values, date_ticks, country, epoch):
    """
    Plot predictions and training results.
    """
    # Ensure the save directory exists
    #if not os.path.exists(save_path):
    #    os.makedirs(save_path)

    fig, ax1 = plt.subplots(figsize=(18, 8))
    plt.title(f"{country}. Epoch: {epoch + 1}")

    # Original Data
    ax1.plot(t_data, y_data, 'go', markersize=8, label='Reported data')

    # Training Data
    ax1.plot(t_data, y_train, 'ro', markersize=8, label='Training data')

    # Predicted values
    ax1.plot(t_predict, predicted_values, color="orange", label="Predicted by PINN")

    # Format x-axis
    ax1.set_xlabel("Date")
    ax1.set_ylabel("I(t)", color="k")
    ax1.tick_params(axis='y', labelcolor="k")
    ax1.legend(loc="upper left")
    ax1.grid()

    # Set date ticks
    tick_labels = date_ticks[:len(t_data[::7])]
    ax1.set_xticks(t_data[::7].flatten())
    ax1.set_xticklabels(tick_labels, rotation=45, ha="right", rotation_mode="anchor")

    # Plotting
    plt.tight_layout()
    plt.show()
    plt.close()